# Сцена 2

![](Scene2.png)

## эффекты: переменная глубина резкости(2 балла)

In [22]:
import math
import numpy as np
from tqdm import tqdm
import matplotlib.pyplot as plt
from PIL import Image

##### Размер окна наблюдателя

In [23]:
d = 10
Vw = 10
Vh = 10

In [24]:
#фон
background_color = (200, 200, 200)
#сферы
center = [ (-2.75, 2.75, 6), 
           (0, 2.75, 9), 
           (2.75, 2.75, 14),
           (-2.75, 0, 7), 
           (0, 0, 10), 
           (2.75, 0, 5),
           (-2.75, -2.75, 11), 
           (0, -2.75, 9.5), 
           (2.75, -2.75, 12) ]
clr = (121, 167, 141) #цвет сферы
r = 1 #радиус

#свет
intensity = (0.2, 0.8)
light_position = (-2, 8, 0)

##### Функции для работы с векторами

In [25]:
def to_vec(A, B):
    return (B[0]-A[0], B[1]-A[1], B[2]-A[2])

#длина вектора
def vector_len(V):
    return math.sqrt(V[0]*V[0] + V[1]*V[1] + V[2]*V[2])

#скалярное произведение вектора
def dot(V1, V2):
    return V1[0]*V2[0] + V1[1]*V2[1] + V1[2]*V2[2]

#изменение направление вектора на противоположное 
def neg_vector(V):
    return (-V[0], -V[1], -V[2])

#сложение двух векторов
def add_vectors(V1, V2):
    return (V1[0]+V2[0], V1[1]+V2[1], V1[2]+V2[2])

#разность двух векторов
def sub_vectors(V1, V2):
    return (V1[0]-V2[0], V1[1]-V2[1], V1[2]-V2[2])

#частное вектора и числа 
def div_vector(V, a):
    return (V[0]/a, V[1]/a, V[2]/a)

In [26]:
#переход в трехмерные координаты
def CanvasToViewport(x, y): 
    return (x*Vw/Cw, -y*Vh/Ch, d)

#вычисление отраженного луча
def ReflectRay(R, N):
    X = (2*N[0]*dot(N,R)-R[0], 2*N[1]*dot(N,R)-R[1], 2*N[2]*dot(N,R)-R[2])
    return X

##### Расстояние от наблюдателя до плоскости фокуса

In [27]:
def F(O, D):
    n = (0, 0, -1) #нормаль к плоскости
    Q = (1, 1, 10) #точка на плоскости
    
    OQ = (Q[0]-O[0], Q[1]-O[1], Q[2]-O[2])
    OD = (D[0]-O[0], D[1]-O[1], D[2]-O[2])
    
    t = dot(n, OQ) / dot(n, OD)
    return t

In [28]:
def IntersectRaySphere(O, D, i):
    C = center[i]
    
    OC = (O[0] - C[0], O[1] - C[1], O[2] - C[2])

    k1 = dot(D, D)
    k2 = 2 * dot(OC, D)
    k3 = dot(OC, OC) - r*r
    
    #решение квадратного уравнения
    discriminant = k2*k2 - 4*k1*k3
    if (discriminant < 0):
        return (math.inf, math.inf) 

    t1 = (-k2 + math.sqrt(discriminant)) / (2*k1)
    t2 = (-k2 - math.sqrt(discriminant)) / (2*k1)
    return (t1, t2)

##### Расчет освещения

In [29]:
def ComputeLighting(P, N, V):
    i = 0.0
    for num in range(0, 2):
        if (num == 0):
            #общее освещение
            i += intensity[0]
        else:    
            #точечный источник света
            L = (light_position[0] - P[0], 
                 light_position[1] - P[1], 
                 light_position[2] - P[2])   
            n_dot_l = dot(N, L)
            if (n_dot_l > 0):
                i += intensity[1] * n_dot_l / (vector_len(N)*vector_len(L)) 

    return i

##### Определение бляжайшего объекта

In [30]:
def ClosestIntersection(O, D, t_min, t_max):
    closest_t = 1000000
    closest_sphere = -1
    tmp = closest_t
    for k in range(0, 9):
        
        t1 = IntersectRaySphere(O, D, k)[0]
        t2 = IntersectRaySphere(O, D, k)[1]
        
        if (t1 > t_min and t1 < t_max and t1 < closest_t):
            tmp = closest_sphere
            closest_t = t1
            closest_sphere = k
        
        if (t2 > t_min and t2 < t_max and t2 < closest_t):
            tmp = closest_t
            closest_t = t2
            closest_sphere = k 
        
    return (closest_sphere, closest_t, tmp)

##### Трассировка луча

In [31]:
def TraceRay(O, D, t_min, t_max, depth): 
    
    closest_sphere = ClosestIntersection(O, D, t_min, t_max)[0]
    closest_t = ClosestIntersection(O, D, t_min, t_max)[1]
    tmp = ClosestIntersection(O, D, t_min, t_max)[2]
    
    if (closest_sphere == -1):
        return background_color
    
    P = (O[0] + closest_t*D[0], O[1] + closest_t*D[1], O[2] + closest_t*D[2])      
    X = (P[0] - center[closest_sphere][0], 
         P[1] - center[closest_sphere][1], 
         P[2] - center[closest_sphere][2] )
    lenN = length(X)
    N = (X[0] / lenN, X[1] / lenN, X[2] / lenN) 
    
    X = (-D[0], -D[1], -D[2])
        
    local_color = (clr[0]*ComputeLighting(P, N, X), 
                   clr[1]*ComputeLighting(P, N, X), 
                   clr[2]*ComputeLighting(P, N, X) )
    return local_color

In [32]:
image = Image.open(r'canvas.png')  
  
Cw, Ch = image.size #размер холста

OO = (0, 0, 0)
O = []
x = 0.1
y = 0

num_of_lights = 16
quarter = 4

for light in range(0, num_of_lights):
    if(light >= 0 and light < quarter):
        x += x/quarter
        y -= y/quarter
    if(light >= quarter and light < quarter*2):
        x -= x/quarter
        y -= y/quarter
    if(light >= quarter*2 and light < quarter*3):
        x -= x/quarter
        y += y/quarter
    if(light >= quarter*3 and light < quarter*4):
        x += x/quarter
        y += y/quarter
    O.append((OO[0]+x, OO[1]+y, OO[2]))  

for x in tqdm(range(-300, 300)):
    x1 = x + 300
    for y in range(-300, 300):
        y1 = y + 300
        s1 = 0
        s2 = 0
        s3 = 0

        X = CanvasToViewport(x, y)
        f = F(OO, X)
        foc = (OO[0] + f*X[0], OO[1] + f*X[1], OO[2] + f*X[2])
        for k in range (0, num_of_lights):
            a = to_vec(OO, O[k]) #O_Oi
            D = sub_vectors(foc, a) 
            Q = TraceRay(O[k], D, 0, math.inf, 1)                    
            color = ( Q[0]/num_of_lights, Q[1]/num_of_lights, Q[2]/num_of_lights)
            s1 += color[0]
            s2 += color[1]
            s3 += color[2]
        s = (s1, s2, s3)
        image.putpixel((x1, y1), tuple(int(c) for c in s) )  
image.show()        

  1%|          | 6/600 [00:07<12:58,  1.31s/it]


KeyboardInterrupt: 